In [ ]:
from smolagents import CodeAgent,DuckDuckGoSearchTool, HfApiModel,load_tool,tool
import datetime
import requests
import pytz
import yaml
from tools.final_answer import FinalAnswerTool
from tools.visit_webpage import VisitWebpageTool

# from Gradio_UI import GradioUI

# Below is an example of a tool that does nothing. Amaze us with your creativity !
@tool
def my_custom_tool(arg1:str, arg2:int)-> str: #it's import to specify the return type
    #Keep this format for the description / args / args description but feel free to modify the tool
    """A tool that does nothing yet 
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"


final_answer = FinalAnswerTool()
visit_webpage = VisitWebpageTool()
# If the agent does not answer, the model is overloaded, please use another model or the following Hugging Face Endpoint that also contains qwen2.5 coder:
# model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud' 

model = HfApiModel(
max_tokens=2096,
temperature=0.5,
# model_id='Qwen/Qwen2.5-Coder-32B-Instruct',# it is possible that this model may be overloaded
model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud',
custom_role_conversions=None,
)


# Import tool from Hub
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)
    
agent = CodeAgent(
    model=model,
    tools=[visit_webpage,image_generation_tool,final_answer], ## add your tools here (don't remove final answer)
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)


# GradioUI(agent).launch()

In [ ]:
from smolagents.agent_types import AgentAudio, AgentImage, AgentText, handle_agent_output_types
from smolagents.agents import ActionStep, MultiStepAgent
from smolagents.memory import MemoryStep
from smolagents.utils import _is_package_available
from typing import Optional
def stream_to_gradio(
    agent,
    task: str,
    reset_agent_memory: bool = False,
    additional_args: Optional[dict] = None,
):
    """Runs an agent with the given task and streams the messages from the agent as gradio ChatMessages."""
    if not _is_package_available("gradio"):
        raise ModuleNotFoundError(
            "Please install 'gradio' extra to use the GradioUI: `pip install 'smolagents[gradio]'`"
        )
    import gradio as gr

    total_input_tokens = 0
    total_output_tokens = 0

    for step_log in agent.run(task, stream=True, reset=reset_agent_memory, additional_args=additional_args):
        # Track tokens if model provides them
        if hasattr(agent.model, "last_input_token_count"):
            total_input_tokens += agent.model.last_input_token_count
            total_output_tokens += agent.model.last_output_token_count
            if isinstance(step_log, ActionStep):
                step_log.input_token_count = agent.model.last_input_token_count
                step_log.output_token_count = agent.model.last_output_token_count

        for message in pull_messages_from_step(
            step_log,
        ):
            yield message

    final_answer = step_log  # Last log is the run's final_answer
    final_answer = handle_agent_output_types(final_answer)

    if isinstance(final_answer, AgentText):
        yield gr.ChatMessage(
            role="assistant",
            content=f"**Final answer:**\n{final_answer.to_string()}\n",
        )
    elif isinstance(final_answer, AgentImage):
        yield gr.ChatMessage(
            role="assistant",
            content={"path": final_answer.to_string(), "mime_type": "image/png"},
        )
    elif isinstance(final_answer, AgentAudio):
        yield gr.ChatMessage(
            role="assistant",
            content={"path": final_answer.to_string(), "mime_type": "audio/wav"},
        )
    else:
        yield gr.ChatMessage(role="assistant", content=f"**Final answer:** {str(final_answer)}")

In [ ]:
def pull_messages_from_step(
    step_log: MemoryStep,
):
    """Extract ChatMessage objects from agent steps with proper nesting"""
    import gradio as gr

    if isinstance(step_log, ActionStep):
        # Output the step number
        step_number = f"Step {step_log.step_number}" if step_log.step_number is not None else ""
        yield gr.ChatMessage(role="assistant", content=f"**{step_number}**")

        # First yield the thought/reasoning from the LLM
        if hasattr(step_log, "model_output") and step_log.model_output is not None:
            # Clean up the LLM output
            model_output = step_log.model_output.strip()
            # Remove any trailing <end_code> and extra backticks, handling multiple possible formats
            model_output = re.sub(r"```\s*<end_code>", "```", model_output)  # handles ```<end_code>
            model_output = re.sub(r"<end_code>\s*```", "```", model_output)  # handles <end_code>```
            model_output = re.sub(r"```\s*\n\s*<end_code>", "```", model_output)  # handles ```\n<end_code>
            model_output = model_output.strip()
            yield gr.ChatMessage(role="assistant", content=model_output)

        # For tool calls, create a parent message
        if hasattr(step_log, "tool_calls") and step_log.tool_calls is not None:
            first_tool_call = step_log.tool_calls[0]
            used_code = first_tool_call.name == "python_interpreter"
            parent_id = f"call_{len(step_log.tool_calls)}"

            # Tool call becomes the parent message with timing info
            # First we will handle arguments based on type
            args = first_tool_call.arguments
            if isinstance(args, dict):
                content = str(args.get("answer", str(args)))
            else:
                content = str(args).strip()

            if used_code:
                # Clean up the content by removing any end code tags
                content = re.sub(r"```.*?\n", "", content)  # Remove existing code blocks
                content = re.sub(r"\s*<end_code>\s*", "", content)  # Remove end_code tags
                content = content.strip()
                if not content.startswith("```python"):
                    content = f"```python\n{content}\n```"

            parent_message_tool = gr.ChatMessage(
                role="assistant",
                content=content,
                metadata={
                    "title": f"🛠️ Used tool {first_tool_call.name}",
                    "id": parent_id,
                    "status": "pending",
                },
            )
            yield parent_message_tool

            # Nesting execution logs under the tool call if they exist
            if hasattr(step_log, "observations") and (
                step_log.observations is not None and step_log.observations.strip()
            ):  # Only yield execution logs if there's actual content
                log_content = step_log.observations.strip()
                if log_content:
                    log_content = re.sub(r"^Execution logs:\s*", "", log_content)
                    yield gr.ChatMessage(
                        role="assistant",
                        content=f"{log_content}",
                        metadata={"title": "📝 Execution Logs", "parent_id": parent_id, "status": "done"},
                    )

            # Nesting any errors under the tool call
            if hasattr(step_log, "error") and step_log.error is not None:
                yield gr.ChatMessage(
                    role="assistant",
                    content=str(step_log.error),
                    metadata={"title": "💥 Error", "parent_id": parent_id, "status": "done"},
                )

            # Update parent message metadata to done status without yielding a new message
            parent_message_tool.metadata["status"] = "done"

        # Handle standalone errors but not from tool calls
        elif hasattr(step_log, "error") and step_log.error is not None:
            yield gr.ChatMessage(role="assistant", content=str(step_log.error), metadata={"title": "💥 Error"})

        # Calculate duration and token information
        step_footnote = f"{step_number}"
        if hasattr(step_log, "input_token_count") and hasattr(step_log, "output_token_count"):
            token_str = (
                f" | Input-tokens:{step_log.input_token_count:,} | Output-tokens:{step_log.output_token_count:,}"
            )
            step_footnote += token_str
        if hasattr(step_log, "duration"):
            step_duration = f" | Duration: {round(float(step_log.duration), 2)}" if step_log.duration else None
            step_footnote += step_duration
        step_footnote = f"""<span style="color: #bbbbc2; font-size: 12px;">{step_footnote}</span> """
        yield gr.ChatMessage(role="assistant", content=f"{step_footnote}")
        yield gr.ChatMessage(role="assistant", content="-----")


In [25]:
meassages=[]
for msg in stream_to_gradio(agent, "plz tell me the main information in this website 'https://github.com/huggingface/diffusers'"):
    meassages.append(msg.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ plz tell me the main information in this website 'https://github.com/huggingface/diffusers'                     │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  url = "https://github.com/huggingface/diffusers"                                                                 
  webpage_content = visit_webpage(url=url)                                                                         
  print(webpage_content)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
GitHub - huggingface/diffusers: 🤗 Diffusers: State-of-the-art diffusion models for image, video, and audio 
generation in PyTorch and FLAX.

[Skip to content](#start-of-content)

Navigation Menu
---------------

Toggle navigation

 

[Sign in](/login?return_to=https%3A%2F%2Fgithub.com%2Fhuggingface%2Fdiffusers)

* Product
  
  
  
  + [GitHub Copilot
    Write better code with AI](https://github.com/features/copilot)
  + [Security
    Find and fix vulnerabilities](https://github.com/features/security)
  + [Actions
    Automate any workflow](https://github.com/features/actions)
  + [Codespaces
    Instant dev environments](https://github.com/features/codespaces)
  + [Issues
    Plan and track work](https://github.com/features/issues)
  + [Code Review
    Manage code changes](https://github.com/features/code-review)
  + [Discussions
    Collaborate outside of code](https://github.com/features/discussions)
  + [Code Search
    Find more, search less](https://github.com/features/code-search)
  
  Explore
  + [All features](https://github.com/features)
  + [Documentation](https://docs.github.com)
  + [GitHub Skills](https://skills.github.com)
  + [Blog](https://github.blog)
* Solutions
  
  
  
  By company size
  + [Enterprises](https://github.com/enterprise)
  + [Small and medium teams](https://github.com/team)
  + [Startups](https://github.com/enterprise/startups)
  + [Nonprofits](/solutions/industry/nonprofits)
  By use case
  + [DevSecOps](/solutions/use-case/devsecops)
  + [DevOps](/solutions/use-case/devops)
  + [CI/CD](/solutions/use-case/ci-cd)
  + [View all use cases](/solutions/use-case)
  
  By industry
  + [Healthcare](/solutions/industry/healthcare)
  + [Financial services](/solutions/industry/financial-services)
  + [Manufacturing](/solutions/industry/manufacturing)
  + [Government](/solutions/industry/government)
  + [View all industries](/solutions/industry)
  
  [View all solutions](/solutions)
* Resources
  
  
  
  Topics
  + [AI](/resources/articles/ai)
  + [DevOps](/resources/articles/devops)
  + [Security](/resources/articles/security)
  + [Software Development](/resources/articles/software-development)
  + [View all](/resources/articles)
  
  Explore
  + [Learning Pathways](https://resources.github.com/learn/pathways)
  + [Events & Webinars](https://resources.github.com)
  + [Ebooks & Whitepapers](https://github.com/resources/whitepapers)
  + [Customer Stories](https://github.com/customer-stories)
  + [Partners](https://partner.github.com)
  + [Executive Insights](https://github.com/solutions/executive-insights)
* Open Source
  
  
  
  + [GitHub Sponsors
    Fund open source developers](/sponsors)
  + [The ReadME Project
    GitHub community articles](https://github.com/readme)
  Repositories
  + [Topics](https://github.com/topics)
  + [Trending](https://github.com/trending)
  + [Collections](https://github.com/collections)
* Enterprise
  
  
  
  + [Enterprise platform
    AI-powered developer platform](/enterprise)
  Available add-ons
  + [Advanced Security
    Enterprise-grade security features](https://github.com/enterprise/advanced-security)
  + [GitHub Copilot
    Enterprise-grade AI features](/features/copilot#enterprise)
  + [Premium Support
    Enterprise-grade 24/7 support](/premium-support)
* [Pricing](https://github.com/pricing)

Search or jump to...

Search code, repositories, users, issues, pull requests...
==========================================================

 
Search

Clear
 

 

[Search syntax 
tips](https://docs.github.com/search-github/github-code-search/understanding-github-code-search-syntax) 

Provide feedback
================

 

We read every piece of feedback, and take your input very seriously.

Include my email address so I can be contacted

  Cancel

 Submit feedback

Saved searches
==============

Use saved searches to filter your results more quickly
------------------------------------------------------

 

Name

Query

To see all available qualifiers, se

[Step 4: Duration 3.99 seconds| Input tokens: 15,252 | Output tokens: 742]

...

[Step 6: Duration 6.36 seconds| Input tokens: 29,574 | Output tokens: 1,295]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import re                                                                                                        
                                                                                                                   
  # Extract the repository description                                                                             
  description_match = re.search(r'🤗 Diffusers: (.+?)\n', webpage_content)                                         
  description = description_match.group(1) if description_match else "No description found"                        
                                                                                                                   
  # Extract any notable sections                                                                                   
  notable_sections = re.findall(r'### (.+?)\n(.*?)(?=\n### |\Z)', webpage_content, re.DOTALL)                      
                                                                                                                   
  # Summarize the main information                                                                                 
  main_information = f"Description: {description}\n"                                                               
  for section_title, section_content in notable_sections:                                                          
      main_information += f"\n### {section_title}\n{section_content.strip()}"                                      
                                                                                                                   
  # Print the main information                                                                                     
  print(main_information)                                                                                          
  final_answer(main_information)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Description: State-of-the-art diffusion models for image, video, and audio generation in PyTorch and FLAX.

### Topics
[flux](/topics/flux "Topic: flux")
[deep-learning](/topics/deep-learning "Topic: deep-learning")
[pytorch](/topics/pytorch "Topic: pytorch")
[image-generation](/topics/image-generation "Topic: image-generation")
[flax](/topics/flax "Topic: flax")
[hacktoberfest](/topics/hacktoberfest "Topic: hacktoberfest")
[diffusion](/topics/diffusion "Topic: diffusion")
[text2image](/topics/text2image "Topic: text2image")
[image2image](/topics/image2image "Topic: image2image")
[jax](/topics/jax "Topic: jax")
[text2video](/topics/text2video "Topic: text2video")
[score-based-generative-modeling](/topics/score-based-generative-modeling "Topic: score-based-generative-modeling")
[stable-diffusion](/topics/stable-diffusion "Topic: stable-diffusion")
[stable-diffusion-diffusers](/topics/stable-diffusion-diffusers "Topic: stable-diffusion-diffusers")
[latent-diffusion-models](/topics/latent-diffusion-models "Topic: latent-diffusion-models")
[video2video](/topics/video2video "Topic: video2video")
[image2video](/topics/image2video "Topic: image2video")
### Resources
[Readme](#readme-ov-file)
### License
[Apache-2.0 license](#Apache-2.0-1-ov-file)
### Code of conduct
[Code of conduct](#coc-ov-file)

[Activity](/huggingface/diffusers/activity) 
[Custom properties](/huggingface/diffusers/custom-properties)
### Stars
[**27.6k**
stars](/huggingface/diffusers/stargazers)
### Watchers
[**214**
watching](/huggingface/diffusers/watchers)
### Forks
[**5.7k**
forks](/huggingface/diffusers/forks) 
[Report 
repository](/contact/report-content?content_url=https%3A%2F%2Fgithub.com%2Fhuggingface%2Fdiffusers&report=huggingfa
ce+%28user%29) 

[Releases
81](/huggingface/diffusers/releases)
----------------------------------------------

[v0.32.2
Latest
 
Jan 15, 2025](/huggingface/diffusers/releases/tag/v0.32.2) 
[+ 80 releases](/huggingface/diffusers/releases)

[Packages
0](/orgs/huggingface/packages?repo_name=diffusers)
------------------------------------------------------------

No packages published   

[Used by 22.4k](/huggingface/diffusers/network/dependents)
----------------------------------------------------------

[* ![@TorNDark](https://avatars.githubusercontent.com/u/70763730?s=64&v=4)
* ![@Haniff-Toha](https://avatars.githubusercontent.com/u/68419391?s=64&v=4)
* ![@Rohan13253](https://avatars.githubusercontent.com/u/142420754?s=64&v=4)
* ![@inho9899](https://avatars.githubusercontent.com/u/80572746?s=64&v=4)
* ![@HyeonjeongHa](https://avatars.githubusercontent.com/u/44867153?s=64&v=4)
* ![@JanPanjan](https://avatars.githubusercontent.com/u/161572470?s=64&v=4)
* ![@PRITHIVSAKTHIUR](https://avatars.githubusercontent.com/u/117709109?s=64&v=4)
* ![@seeward](https://avatars.githubusercontent.com/u/1984188?s=64&v=4)

+ 22,365](/huggingface/diffusers/network/dependents)

[Contributors
911](/huggingface/diffusers/graphs/contributors)
--------------------------------------------------------------

[+ 897 contributors](/huggingface/diffusers/graphs/contributors)

Languages
---------

* [Python
  99.9%](/huggingface/diffusers/search?l=python)
* Other
  0.1%

Footer
------

© 2025 GitHub, Inc.
### Footer navigation
* [Terms](https://docs.github.com/site-policy/github-terms/github-terms-of-service)
* [Privacy](https://docs.github.com/site-policy/privacy-policies/github-privacy-statement)
* [Security](https://github.com/security)
* [Status](https://www.githubstatus.com/)
* [Docs](https://docs.github.com/)
* [Contact](https://support.github.com?tags=dotcom-footer)
* Manage cookies
* Do not share my personal information

You can’t perform that action at this time.

Out - Final answer: Description: State-of-the-art diffusion models for image, video, and audio generation in 
PyTorch and FLAX.

### Topics
[flux](/topics/flux "Topic: flux")
[deep-learning](/topics/deep-learning "Topic: deep-learning")
[pytorch](/topics/pytorch "Topic: pytorch

[Step 7: Duration 6.89 seconds| Input tokens: 38,684 | Output tokens: 1,512]